In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud_with_ctor_args import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "weakPassword"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GS_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Brian Baecher\'s CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'100px'}),
    html.Hr(),
    html.Div(
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    className="buttonRow",
        style={"display": "flex"},
        children = [
            html.Button(id="submit-button-water", n_clicks=0, children="Water Rescue Animals"),
            html.Button(id="submit-button-wilderness", n_clicks=0, children="Wilderness Rescue Animals"),
            html.Button(id="submit-button-disaster", n_clicks=0, children="Disaster Rescue Animals"),
            html.Button(id="submit-button-reset", n_clicks=0, children="Reset Table")
        ]
    ),
    # adding another div, for a label showing active filter if any
    # updated in the button callback
    html.Div(id='filter-label', style={"fontWeight": "bold"}),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        style_table={'overflowX': 'auto'},
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '100px', 'maxWidth': '180px'},
        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#Water Rescue
#Mountain or Wilderness Rescue
#Disaster or Individual Tracking
#Reset (returns all widgets to their original, unfiltered state)


# added label to output
@app.callback([Output('datatable-id','data'),
              Output('filter-label', 'children')],
              [Input('submit-button-water', 'n_clicks'),
              Input('submit-button-wilderness', 'n_clicks'),
              Input('submit-button-disaster', 'n_clicks'),
              Input('submit-button-reset', 'n_clicks')])
def update_dashboard(a1, a2, a3, a4):
    # the args are required? it's just passing in the n_clicks which I'm not going to use.
    #https://dash.plotly.com/advanced-callbacks
    # scroll to properties for callback_context section
    # should allow for me to get id of button clicked, rather than by comparing n_clicks
    ctx = dash.callback_context
    if not ctx.triggered:
        return df.to_dict('records'), "No Filter Applied"
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0] #getting button id string
    
    if button_id == "submit-button-water":
        water_query = {
    "breed": { "$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"] },
    "sex_upon_outcome": "Intact Female",
    "age_upon_outcome_in_weeks": { "$gte": 26, "$lte": 156 },
    # the filter below wasn't included in the table that rubric provided for search terms, 
    # but it doesn't make much sense to include dead animals in any of these queries.
    "outcome_type": {"$nin": ["Euthanasia", "Died", "Disposal"]} 
    }
        label_string = "Water Rescue"
        filtered = db.read(water_query)
    elif button_id == "submit-button-wilderness": 
        mountain_query = {
    "breed": { "$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"] },
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": { "$gte": 26, "$lte": 156 },
    "outcome_type": {"$nin": ["Euthanasia", "Died", "Disposal"]}
    }
        label_string = "Mountain or Wilderness Rescue"
        filtered = db.read(mountain_query)
    elif button_id == "submit-button-disaster":
        disaster_query = {
    "breed": { "$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"] },
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 300 },
    "outcome_type": {"$nin": ["Euthanasia", "Died", "Disposal"]}
    }
        label_string = "Disaster or Individual Tracking"
        filtered = db.read(disaster_query)
    else:
        # the only other caller to this is the reset button, just get all.
        label_string = "No Filter Applied"
        filtered = db.read({})
    
    # drop ObjectId column
    data = pd.DataFrame.from_records(filtered).drop(columns=['_id']).to_dict('records')
    return data, label_string

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    # gonna try a bar chart for top 5 breeds of selected animal_type (or both if none selected)
    if viewData is None:
        return []
    
    frame = pd.DataFrame(viewData)
    breed_cts = frame['breed'].value_counts().nlargest(5).reset_index()
    breed_cts.columns = ['breed', 'count']
    
    bar_figure = px.bar(breed_cts, x='breed', y='count', title='Top 5 Breeds', text='count')
    bar_figure.update_layout(xaxis_title='Breed', yaxis_title="Count")
    
    return [dcc.Graph(figure=bar_figure)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')] # changed to rows
)
def update_styles(selected_rows):# changed arg to rows
    # the provided code for this function throws error. In Mod6 I fixed it by styling rows, not columns.
    # pasting in my solution from that assignment
    if selected_rows is None or len(selected_rows) == 0:
        return []
    
    return [{
        'if': { 'row_index': selected_rows[0] },
        'background_color': '#D2F3FF'
    }]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:17444/
